In [1]:
!python3 -m ipykernel install --user --name=env

Installed kernelspec env in /h/u14/c2/00/saragihd/.local/share/jupyter/kernels/env


In [3]:
# Load save 
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import models
from models.quantization import quan_Conv2d, quan_Linear, quantize
from utils import AverageMeter, RecorderMeter

In [4]:
# Load mnist
root = './data'
download = True
train_set = dset.MNIST(root=root, train=True, transform=None, download=download)
test_set = dset.MNIST(root=root, train=False, transform=None, download=download)

In [5]:
# Show data
def show(img):
    # Show image
    # Show with PIL
    plt.imshow(img, cmap='gray')
    print("Showing")
    plt.show()

# Get image
img, label = train_set[0]
print('Label:', label)
print('Image Size:', img.size)

# Convert to numpy
img = np.array(img)
print('Numpy Shape:', img.shape)

# Show image
show(img)

Label: 5
Image Size: (28, 28)
Numpy Shape: (28, 28)
Showing


In [6]:
# Add colour jitter, random affine, random horizontal flip, random rotation, gaussian blur, random erasing, and normalization
mean = (0.5,)
std = (0.5,)
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(p=0.8),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(3),
    transforms.RandomErasing(p=0.9, scale=(0.02, 0.1)),
])

# Apply transform
img, label = train_set[0]
img = transform(img)
print('Transformed Shape:', img.size())

# Show image
img = img.permute(1, 2, 0)
img = img.numpy()
show(img)

Transformed Shape: torch.Size([1, 32, 32])
Showing


In [10]:
DATASET = 'finetune_mnist'
data_path = './data'
ARCH = "resnet32_quan"
chk_path = './save_finetune/cifar60/model_best.pth.tar'
BATCH_SIZE = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'
use_cuda = torch.cuda.is_available()

if DATASET == 'finetune_mnist':
    mean = [0.5]
    std = [0.5]
    test_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize(mean, std)
    ])
if DATASET == 'finetune_mnist':
    test_data = dset.MNIST(data_path,
                            train=False,
                            transform=test_transform,
                            download=True)
    num_classes = 10
    num_channels = 3

test_loader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                        num_workers=1,
                                        pin_memory=True)

In [11]:
net = models.__dict__[ARCH](num_classes, num_channels)

checkpoint = torch.load(chk_path)
state_tmp = net.state_dict()
if 'state_dict' in checkpoint.keys():
    state_tmp.update(checkpoint['state_dict'])
else:
    state_tmp.update(checkpoint)

#net.load_state_dict(state_tmp)
model_dict = net.state_dict()
pretrained_dict = {k:v for k, v in checkpoint['state_dict'].items() if k in model_dict}
model_dict.update(pretrained_dict)
net.load_state_dict(model_dict)

CifarResNet : Depth : 32 , Layers for each block : 5
self.b_index: 0
self.b_index: 1
self.b_index: 2
self.b_index: 3
self.b_index: 4
self.b_index: 5
self.b_index: 6
self.b_index: 7
self.b_index: 8
self.b_index: 9
self.b_index: 10
self.b_index: 11
self.b_index: 12
self.b_index: 13
self.b_index: 14


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
def accuracy(output, target, topk=(1, )):
    """Computes the accuracy@k for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            #correct_k = correct[:k].view(-1).float().sum(0)
            correct_k = correct[:k].reshape(-1).float().sum(0)
            
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validate(val_loader, model, num_branch, ic_only):
    top1 = AverageMeter()
    top5 = AverageMeter()

    print("Validating...")

    top1_list = []
    for idx in range(num_branch):
        top1_list.append(AverageMeter())
    top5_list = []
    for idx in range(num_branch):
        top5_list.append(AverageMeter())

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                target = target.cuda(non_blocking=True)
                input = input.cuda()

            # compute output
            output_branch = model(input)
            # measure accuracy and record loss
            #prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
            
            for idx in range(len(output_branch)):
                prec1, prec5 = accuracy(output_branch[idx].data, target, topk=(1, 5))
                top1_list[idx].update(prec1, input.size(0)) 
                top5_list[idx].update(prec5, input.size(0))
        
        print(
        '  **Test** Prec_B1@1 {top1_b1.avg:.3f} Prec_B1@5 {top5_b1.avg:.3f} Error@1 {error1:.3f}'
        '  **Test** Prec_B2@1 {top1_b2.avg:.3f} Prec_B2@5 {top5_b2.avg:.3f} Error@1 {error2:.3f}'
        '  **Test** Prec_B3@1 {top1_b3.avg:.3f} Prec_B3@5 {top5_b3.avg:.3f} Error@1 {error3:.3f}'
        '  **Test** Prec_B4@1 {top1_b4.avg:.3f} Prec_B4@5 {top5_b4.avg:.3f} Error@1 {error4:.3f}'
        '  **Test** Prec_B5@1 {top1_b5.avg:.3f} Prec_B5@5 {top5_b5.avg:.3f} Error@1 {error5:.3f}'
        '  **Test** Prec_B6@1 {top1_b6.avg:.3f} Prec_B6@5 {top5_b6.avg:.3f} Error@1 {error6:.3f}'
        '  **Test** Prec_Bmain@1 {top1_main.avg:.3f} Prec_Bmain@5 {top5_main.avg:.3f} Error@1 {errormain:.3f}'
        .format(top1_b1=top1_list[0], top5_b1=top5_list[0], error1=100 - top1_list[0].avg,
                top1_b2=top1_list[1], top5_b2=top5_list[1], error2=100 - top1_list[1].avg,
                top1_b3=top1_list[2], top5_b3=top5_list[2], error3=100 - top1_list[2].avg,
                top1_b4=top1_list[3], top5_b4=top5_list[3], error4=100 - top1_list[3].avg,
                top1_b5=top1_list[4], top5_b5=top5_list[4], error5=100 - top1_list[4].avg,
                top1_b6=top1_list[5], top5_b6=top5_list[5], error6=100 - top1_list[5].avg,
                top1_main=top1_list[-1], top5_main=top5_list[-1], errormain=100 - top1_list[-1].avg,
        ))
        
    sum=0
    if ic_only: #if only train ic branch
        for item in top1_list[:-1]:
            sum += item.avg
        top1_avg = sum/len(top1_list[:-1])
        sum=0
        for item in top5_list[:-1]:
            sum += item.avg
        top5_avg = sum/len(top5_list[:-1])
    else:
        top1_avg = top1_list[-1].avg
        top5_avg = top5_list[-1].avg

    return top1_avg, top5_avg

In [ ]:
net.eval()
output_branch = net(input)
num_branch = len(output_branch) # the number of branches
val_acc, _, val_los = validate(test_loader, net, num_branch, ic_only=True)